<a href="https://colab.research.google.com/github/het8802/Fifa_Data_Analysis/blob/main/Fifa_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/het8802/Fifa_Data_Analysis/main/fifa_data.csv')
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,...,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,...,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,...,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,...,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,...,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18207 non-null  int64  
 1   ID                        18207 non-null  int64  
 2   Name                      18207 non-null  object 
 3   Age                       18207 non-null  int64  
 4   Photo                     18207 non-null  object 
 5   Nationality               18207 non-null  object 
 6   Flag                      18207 non-null  object 
 7   Overall                   18207 non-null  int64  
 8   Potential                 18207 non-null  int64  
 9   Club                      17966 non-null  object 
 10  Club Logo                 18207 non-null  object 
 11  Value                     18207 non-null  object 
 12  Wage                      18207 non-null  object 
 13  Special                   18207 non-null  int64  
 14  Prefer

In [4]:
df.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [5]:
unwanted_cols = ['Unnamed: 0', 'ID', 'Photo', 'Nationality', 'Flag', 'Club Logo', 'Joined', 'Loaned From', 'Contract Valid Until', 'Club', 'Jersey Number']
df.drop(unwanted_cols, axis = 1, inplace = True)

In [6]:
def convert_to_total(str) :
    number_list = [int(i) for i in str.split('+')]
    sum = 0
    for i in number_list :
        sum += i
    return sum

In [7]:
to_change_columns = ['LS', 'ST', 'RS', 'LF', 'CF', 'RF', 'RW', 'LW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM','CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB']

In [8]:
for col in to_change_columns :
    final_list = []
    for i in df[col] :

        if type(i) == float :
            final_list.append(np.nan)

        if type(i) == str :
            total = convert_to_total(i)
            final_list.append(total)
    
    df[col] = pd.Series(final_list)

In [9]:
def change_value(i) :
    letter_value = {
        'B' : 1000000000,
        'M' : 1000000,
        'K' : 1000
    }
    if i[-1] in letter_value.keys() :
        num = float(i[:-1].strip())
        letter = i[-1]
        total_num = num * letter_value[letter]
        return total_num
    else :
        return float(i)

In [10]:
def convert_columns(col) :
    vals = []
    for i in df[col] :
        if type(i) == float :
            vals.append(np.nan)
        elif type(i) == str :
            i = i.replace('€', '')
            value = change_value(i.strip())
            vals.append(value)
    
    df[col] = pd.Series(vals)

In [11]:
df['Release Clause']

0        €226.5M
1        €127.1M
2        €228.1M
3        €138.6M
4        €196.4M
          ...   
18202      €143K
18203      €113K
18204      €165K
18205      €143K
18206      €165K
Name: Release Clause, Length: 18207, dtype: object

In [12]:
for i in ['Wage', 'Value', 'Release Clause'] :
    convert_columns(i)

In [13]:
def change_column_values(col) :
    indices = list(df[col].value_counts().index)            # getting all value names
    col_change = dict(zip(indices, range(len(indices))))
    return col_change

In [14]:
pf_change = change_column_values('Preferred Foot')
df['Preferred Foot'].replace(pf_change, inplace = True)

In [15]:
rf_change = change_column_values('Real Face')                       # changing real face
df['Real Face'].replace(rf_change, inplace = True)

In [16]:
col = 'Work Rate'

wr_change = {
    'Low' : 0,
    'Medium' : 1,
    'High' : 2
}
new_col_1 = []
new_col_2 = []

for i in df[col] :
    if type(i) == float :
        new_col_1.append(np.nan)
        new_col_2.append(np.nan)
    elif type(i) == str :
        split_col = [j.strip() for j in i.split('/')]
        new_col_1.append(wr_change[split_col[0]])
        new_col_2.append(wr_change[split_col[1]])

df.insert(13, 'Work Rate 1', pd.Series(new_col_1))
df.insert(14, 'Work Rate 2', pd.Series(new_col_2))
df.drop('Work Rate', axis = 1, inplace = True)

In [17]:
bt_change = {                   # changing body type
    'Normal' : 0,
    'Lean' : 1,
    'Stocky' : 2,
    'Courtois' : 3,                         # giving number 3 to all legendary players
    'Messi' : 3,
    'Neymar' : 3,
    'C. Ronaldo' : 3,
    'Shaqiri' : 3,
    'Akinfenwa' : 3,
    'PLAYER_BODY_TYPE_25' : 3               # the player is Mohamed Salah
}

df['Body Type'].replace(bt_change, inplace = True)

In [18]:
pos_change = change_column_values('Position')
df['Position'].replace(pos_change, inplace = True)

In [19]:
col = 'Height'

indices = list(df[col].value_counts().index)
h_change = {}
for i in indices :
    lst = [int(j) for j in i.split("'")]
    num = round(lst[0] + lst[1]/12, 2)
    h_change[i] = num

df[col].replace(h_change, inplace = True)

In [20]:
indices = list(df['Weight'].value_counts().index)
w_change = {}
for i in indices :
    w_change[i] = int(i.replace('lbs', ''))

df['Weight'].replace(w_change, inplace = True)

In [21]:
# will drop the rows containing NaN values in Release cluase column. And will save the dropped rows so as to test it later.
no_rc_rows = df[df['Release Clause'].isnull()]
df = df.dropna(subset = ['Release Clause'])
no_rc_rows.drop('Release Clause', inplace = True, axis = 1)

# **Now starting to analyse the data**

In [22]:
corr_matrix = df.corr()
rc_corr = corr_matrix['Release Clause'].sort_values(ascending = False)
for i, ind in zip(rc_corr, rc_corr.index) :
    print(ind, ' '*(20 - len(ind)), round(i,2))

Release Clause        1.0
Value                 0.99
Wage                  0.86
International Reputation  0.64
Overall               0.62
Potential             0.59
Reactions             0.53
LCM                   0.49
CM                    0.49
RCM                   0.49
LAM                   0.46
RAM                   0.46
CAM                   0.46
LM                    0.46
RM                    0.46
LS                    0.46
ST                    0.46
RS                    0.46
RF                    0.45
CF                    0.45
LF                    0.45
Real Face             0.45
Composure             0.44
RW                    0.44
LW                    0.44
Special               0.37
LWB                   0.36
RWB                   0.36
Vision                0.35
LDM                   0.34
CDM                   0.34
RDM                   0.34
ShortPassing          0.32
RB                    0.31
LB                    0.31
Skill Moves           0.31
BallControl           0.3

In [23]:
# from google.colab import widgets
# col_lst = list(df.columns)[2:-1]
# tb = widgets.TabBar(col_lst)
# j = 0
# for i in col_lst :
#     with tb.output_to(j) :
#         plt.figure(figsize = (20,6))
#         sns.scatterplot(y = 'Release Clause', x = i, data = df)
        
#         plt.show()
#     j+=1

In [24]:
# cols = list(df.columns[2:])
# tb = widgets.TabBar(cols)
# j = 0

# for i in cols :
#     with tb.output_to(j) :
#         plt.figure(figsize = (20,13))
#         sns.histplot(data = df, x = i)
#         plt.show()
#         j+=1

# **Splitting the data**

In [25]:
from sklearn.impute import SimpleImputer
data = df.iloc[:, 1:]


In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

my_pipeline = Pipeline([
                        ('imputer', SimpleImputer(strategy = 'median')),
                        ('std_scaler', StandardScaler())
])

In [27]:
data = my_pipeline.fit_transform(data)
data = pd.DataFrame(data, columns = df.columns[1:])
data

,Age,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Body Type,Work Rate 1,Work Rate 2,Real Face,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,1.224180,3.972193,3.716187,18.889634,24.946580,2.200160,1.832159,9.748346,1.601850,2.183663,4.099305,-0.414842,-0.178270,3.169195,3.630511,-1.662082,-0.448621,3.506666,3.506666,3.506666,3.524600,3.675016,3.675016,3.675016,3.524600,3.673908,3.673908,3.673908,3.554998,3.089254,3.089254,3.089254,3.554998,0.750883,0.416124,0.416124,0.416124,0.750883,0.220634,-0.813004,-0.813004,-0.813004,0.220634,1.864404,2.545911,1.023382,2.123931,2.441998,2.204082,2.490897,2.920422,2.232619,2.255869,1.765474,1.453487,1.864816,3.642633,2.191081,1.718789,0.242862,0.550303,-0.503893,2.439461,-0.454851,-1.195295,2.258488,2.872055,1.691432,3.244788,-0.718726,-0.912142,-0.927542,-0.602719,-0.323818,-0.081598,-0.145992,-0.488513,19.959280
1,1.648228,3.972193,3.716187,13.033458,17.759719,2.294460,-0.545804,9.748346,1.601850,3.506707,4.099305,1.506730,-2.170853,3.169195,-1.144469,1.014786,1.092325,3.970278,3.970278,3.970278,3.310585,3.460267,3.460267,3.460267,3.310585,3.241425,3.241425,3.241425,3.327013,2.850060,2.850060,2.850060,3.327013,0.986277,0.520961,0.520961,0.520961,0.986277,0.553566,-0.180000,-0.180000,-0.180000,0.553566,1.864404,2.494729,2.110912,1.516246,2.498367,1.730598,1.841076,1.894604,1.582928,2.136712,1.632521,1.792276,1.594741,3.752289,0.429601,2.296551,2.520165,1.546327,1.092235,2.387738,0.406136,-0.857809,2.309481,2.025267,2.325686,3.158125,-0.969521,-0.773773,-1.068423,-0.546639,-0.323818,-0.081598,-0.145992,-0.322679,11.019132
2,0.164060,3.686806,3.553618,20.288124,12.594163,1.986171,-0.545804,9.748346,3.117202,3.506707,4.099305,1.506730,-0.178270,3.169195,1.529520,-0.890781,-1.026476,3.158956,3.158956,3.158956,3.310585,3.352892,3.352892,3.352892,3.310585,3.349546,3.349546,3.349546,3.327013,2.850060,2.850060,2.850060,3.327013,0.986277,0.416124,0.416124,0.416124,0.986277,0.442589,-0.722575,-0.722575,-0.722575,0.442589,1.593446,2.136454,0.565475,1.718807,2.329260,2.151473,2.220138,2.521493,1.647897,2.196291,1.964904,1.724518,2.202408,3.532978,1.416030,1.429908,-0.347549,1.110566,-1.301958,1.818794,0.004342,-0.520322,2.003524,2.378095,2.071984,3.071461,-1.019680,-1.096634,-0.598820,-0.434479,-0.441234,-0.081598,-0.087687,-0.322679,20.103186
3,0.376084,3.544112,3.553618,12.159402,11.246627,-0.451131,-0.545804,7.239234,0.086498,-1.785470,0.846946,-0.414842,-0.178270,3.169195,-0.953470,1.740717,0.129234,0.029569,0.029569,0.029569,0.100359,0.131658,0.131658,0.131658,0.100359,0.105925,0.105925,0.105925,0.135219,-0.020272,-0.020272,-0.020272,0.135219,0.044701,0.101612,0.101612,0.101612,0.044701,0.109657,0.181717,0.181717,0.181717,0.109657,-1.766444,-1.651019,-1.781301,-0.576893,-1.672940,-1.952055,-1.408031,-1.353821,-0.106270,-0.961377,-0.494730,-0.443734,-0.228260,3.094357,-1.472798,-1.401126,0.158518,-1.254990,-0.104861,-1.801763,-1.028842,-0.809596,-1.922933,1.037348,-0.528454,0.818216,-1.621587,-1.235003,-1.538027,4.107983,4.020589,4.251792,4.168576,4.265379,12.053455
4,0.376084,3.544112,3.391048,17.403739,15.513825,2.486688,-0.545804,7.239234,3.117202,2.183663,-0.779233,1.506730,1.814312,3.169195,1.338521,-0.119480,-0.769652,2.927150,2.927150,2.927150,3.096570,3.138143,3.138143,3.138143,3.096570,3.241425,3.241425,3.241425,3.327013,3.567643,3.567643,3.567643,3.327013,2.398641,2.198358,2.198358,2.198358,2.398641,1.885293,0.995579,0.995579,0.995579,1.885293,2.352130,1.880544,0.164806,2.258972,2.216522,1.625380,2.0

In [28]:
data.describe()

,Age,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Body Type,Work Rate 1,Work Rate 2,Real Face,Position,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
count,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04,1.664300e+04
mean,6.665487e-15,-3.573260e-14,-2.731270e-14,1.639015e-14,-2.914807e-14,2.065630e-15,6.796622e-16,4.954081e-14,-9.014557e-15,6.515241e-15,-1.206830e-15,1.318114e-14,-9.111961e-16,3.706766e-14,1.287439e-14,3.751931e-16,2.842412e-15,2.209823e-14,2.209823e-14,2.209823e-14,-8.413800e-14,-1.873883e-14,-1.873883e-14,-1.873883e-14,-8.413800e-14,-2.992875e-14,-2.992875e-14,-2.992875e-14,-1.705113e-15,2.445559e-15,2.445559e-15,2.445559e-15,-1.705113e-15,-1.980928e-14,5.006523e-15,5.006523e-15,5.006523e-15,-1.980928e-14,2.289961e-14,-5.040978e-15,-5.040978e-15,-5.040978e-15,2.289961e-14,5.162250e-15,6.209124e-16,-6.238232e-15,-2.087375e-14,-1.832805e-15,2.070660e-15,-7.258376e-16,-5.927416e-16,5.324621e-15,-4.495226e-15,-2.132208e-15,-3.276896e-15,1.089390e-15,4.557661e-14,1.435080e-15,-5.346788e-15,2.156223e-15,1.043408e-15,-4.570896e-15,5.342045e-15,-4.475054e-15,-9.010429e-15,-6.746044e-15,-1.716281e-14,-2.355834e-14,3.749716e-15,-8.292785e-16,5.629104e-15,6.108128e-16,-9.301845e-16,3.030805e-15,-1.401150e-15,2.719223e-16,8.470195e-16,6.579284e-15
std,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00
min,-1.956181e+00,-2.877097e+00,-3.761999e+00,-4.252574e-01,-3.871040e-01,-3.135064e+00,-5.458042e-01,-2.881040e-01,-2.944206e+00,-1.785470e+00,-7.792333e-01,-2.336413e+00,-2.170853e+00,-3.155376e-01,-1.144469e+00,-3.930615e+00,-3.594720e+00,-3.099817e+00,-3.099817e+00

In [29]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size = 0.2, random_state = 42)

In [30]:
x_train = train_set.iloc[:, :-1]
y_train = train_set.iloc[:, -1]

x_test = test_set.iloc[:, :-1]
y_test = test_set.iloc[:, -1]

In [31]:
x_train.isnull().sum().sum()

0

In [32]:
precision = {}

def add_values(regressor_name, score, rmse) :
    precision[regressor_name] = {
    'score' : score,
    'rmse' : rmse
}

In [33]:
from sklearn.ensemble import RandomForestRegressor

rf_clf = RandomForestRegressor(n_jobs = -1, n_estimators = 50)

In [34]:
rf_clf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [35]:
predicted = rf_clf.predict(x_test)

In [36]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predicted)
rf_rmse = np.sqrt(mse)
rf_rmse

0.13249175293072177

In [37]:
rf_score = rf_clf.score(x_test, y_test)

add_values(regressor_name = 'RandomForest', score = rf_score, rmse = rf_rmse)

In [38]:
# new_test_data = no_rc_rows.iloc[:, 1:]
# test1 = my_pipeline.fit_transform(new_test_data)

# pred2 = rf_clf.predict(test1)

# for i,name in zip(pred2, no_rc_rows['Name']) :
#     print(name, ' ' * (30 - len(name)), i)

In [39]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
lr_score = lr_model.score(x_test, y_test)
lr_score

0.9892929883813003

In [41]:
lr_predicted = lr_model.predict(x_test)
lr_mse = mean_squared_error(y_test, lr_predicted)
lr_rmse = np.sqrt(lr_mse)
lr_rmse

0.11386800081161035

In [42]:
add_values(regressor_name = 'Linear', score = lr_score, rmse = lr_rmse)

In [43]:
from sklearn.tree import DecisionTreeRegressor

dtr_model = DecisionTreeRegressor()
dtr_model.fit(x_train, y_train)

dtr_score = dtr_model.score(x_test, y_test)

dtr_pred = dtr_model.predict(x_test)
dtr_mse = mean_squared_error(y_test, dtr_pred)
dtr_rmse = np.sqrt(dtr_mse)
dtr_rmse

0.14986215071780368

In [44]:
add_values(regressor_name = 'Decision Tree', score = dtr_score, rmse = dtr_rmse)

In [45]:
precision

{'Decision Tree': {'rmse': 0.14986215071780368, 'score': 0.9814540619653077},
 'Linear': {'rmse': 0.11386800081161035, 'score': 0.9892929883813003},
 'RandomForest': {'rmse': 0.13249175293072177, 'score': 0.9855041871113627}}

In [46]:
test_pred = lr_model.predict(np.array(no_rc_rows.iloc[0, 1:]).reshape(1,-1))
no_rc_rows.iloc[0,0], test_pred

('J. Rodríguez', array([70252968.70263368]))

In [47]:
test_pred_2 = lr_model.predict(np.array(df.iloc[0, 1:-1]).reshape(1,-1))

df.iloc[0, -1], test_pred_2

(226500000.0, array([1.1169689e+08]))

In [48]:
lr_predicted[0], 

(0.3941709966703777,)

In [49]:
for i, n in zip(lr_predicted, y_test) :
    print(round(i,2), ' '*(30 - len(str(round(i,2)))), round(n,2))

0.39                            0.35
0.3                             0.62
-0.19                           -0.21
-0.23                           -0.29
-0.31                           -0.32
-0.18                           -0.19
-0.13                           -0.12
-0.33                           -0.33
-0.36                           -0.34
-0.05                           -0.01
-0.36                           -0.33
-0.35                           -0.32
-0.25                           -0.29
-0.31                           -0.3
-0.36                           -0.4
-0.31                           -0.28
-0.35                           -0.36
1.68                            1.85
0.04                            0.04
-0.28                           -0.3
-0.36                           -0.38
-0.37                           -0.37
0.51                            0.61
-0.34                           -0.34
-0.26                           -0.23
0.04                            0.1
-0.4                  